In [108]:
### Data handling imports
import pandas as pd
import numpy as np

import reverse_geocoder as rg
import unidecode

### Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
import copy

sns.set()
%matplotlib inline
%load_ext lab_black
import os
from collections import Counter
from collections import OrderedDict
from operator import itemgetter

import json

# Statistics imports
import scipy, scipy.stats

pd.set_option("display.max_columns", None)

import warnings

warnings.filterwarnings("ignore")


class color:
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"
    END = "\033[0m"

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [2]:
inputFolder = "../twitter-data-geo-output"
outputFolder = "./twitter-data-demographics-output"

In [3]:
def read_json(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

In [4]:
sampleDataPath = outputFolder + "/cleanedSet.json"

In [5]:
sampleData  = read_json(sampleDataPath)
sampleData['0']

{'created_at': 'Thu Mar 19 10:59:57 +0000 2020',
 'id': 1240593849694605312,
 'id_str': '1240593849694605312',
 'text': 'current weather in Burlington: overcast clouds, 36°F\n69% humidity, wind 5mph, pressure 1027mb',
 'source': '<a href="http://twitter.com/USACities/cities" rel="nofollow">Cities</a>',
 'truncated': False,
 'geo': {'type': 'Point', 'coordinates': [44.48, -73.21]},
 'coordinates': [-73.21, 44.48],
 'place': {'id': '8f07721fe6eb5c41',
  'url': 'https://api.twitter.com/1.1/geo/id/8f07721fe6eb5c41.json',
  'place_type': 'city',
  'name': 'Burlington',
  'full_name': 'Burlington, VT',
  'country_code': 'US',
  'country': 'United States',
  'bounding_box': {'type': 'Polygon',
   'coordinates': [[[-73.276995, 44.445714],
     [-73.276995, 44.539485],
     [-73.17455, 44.539485],
     [-73.17455, 44.445714]]]},
  'attributes': {}},
 'contributors': None,
 'is_quote_status': False,
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'entities': {'h

In [6]:
test = rg.search(sampleData['0']['geo']['coordinates'])

Loading formatted geocoded file...


In [7]:
print(test)

[{'lat': '44.47588', 'lon': '-73.21207', 'name': 'Burlington', 'admin1': 'Vermont', 'admin2': 'Chittenden County', 'cc': 'US'}]


In [8]:
df_sample = pd.read_json(sampleDataPath, orient='index')

In [9]:
df_sample.head()

,created_at,id,id_str,text,source,truncated,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,user_id,user_id_str,user_name,user_screen_name,user_location,user_url,user_description,user_translator_type,user_protected,user_verified,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_statuses_count,user_created_at,user_utc_offset,user_time_zone,user_geo_enabled,user_lang,user_contributors_enabled,user_is_translator,user_default_profile,user_default_profile_image,user_following,user_follow_request_sent,user_notifications,display_text_range,extended_entities,possibly_sensitive,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,2020-03-19 10:59:57+00:00,1240593849694605312,1240593849694605312,current weather in Burlington: overcast clouds...,"<a href=""http://twitter.com/USACities/cities"" ...",False,"{'type': 'Point', 'coordinates': [44.48, -73.21]}","[-73.21, 44.48]","{'id': '8f07721fe6eb5c41', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2020-03-19 10:59:57.543,112034664,112034664,Burlington Weather,BurlingtonVT,"Burlington, VT",None,"Weather updates, forecast, warnings and inform...",none,False,False,120,1,14,0,37409,2010-02-07 01:44:27+00:00,NaN,NaN,True,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-19 10:59:59+00:00,1240593857349218309,1240593857349218304,"Flash Flood Warning continues for Nixa MO, Oza...","<a href=""http://www.weather.gov/crh/impact"" re...",False,"{'type': 'Point', 'coordinates': [36.82104014,...","[-93.47691589, 36.82104014]","{'id': '2526edd24c06e60c', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2020-03-19 10:59:59.368,600227042,600227042,NWS Springfield,NWSSpringfield,"Springfield, Missouri",http://www.weather.gov/springfield,Official Twitter account for the National Weat...,none,False,True,33227,306,512,2396,17173,2012-06-05 16:24:58+00:00,NaN,NaN,True,NaN,False,False,False,False,NaN,NaN,NaN,"[0, 83]","{'media': [{'id': 1240593855927300098, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN
2,2020-03-19 11:00:00+00:00,1240593862382489600,1240593862382489600,ee69b5dca7d2458d2039dc8f3e12c093a733e7b83d0618...,"<a href=""https://twitter.com/googuns_lulz"" rel...",True,"{'type': 'Point', 'coordinates': [27.26026626,...","[-166.01094411, 27.26026626]",None,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,fr,2020-03-19 11:00:00.568,3048544857,3048544857,GooGuns Lulz,googuns_lulz,(here),https://victorz.ca/bots/googuns,@victor_zheng,none,False,False,259,1,257,0,3973912,2015-02-21 03:26:23+00:00,NaN,NaN,True,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,{'full_text': 'ee69b5dca7d2458d2039dc8f3e12c09...,NaN,NaN,NaN,NaN
3,2020-03-19 11:00:00+00:00,1240593862659149824,1240593862659149824,"Wind 0.0 mph NNW. Barometer 29.83 in, Rising s...","<a href=""http://sandaysoft.com/"" rel=""nofollow...",False,"{'type': 'Point', 'coordinates': [37.92583333,...","[-120.63, 37.92583333]","{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2020-03-19 11:00:00.634,865368865,865368865,Donald Price,DonaldPrice11,None,None,None,none,False,False,5,24,0,14,8142,2012-10-06 21:52:04+00:00,NaN,NaN,True,NaN,False,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-19 11:00:02+00:00,1240593868438917126,1240593868438917120,"MED - MEDICAL at 6300 SE DIVISION ST, PORT [Po...","<a href=""http://www.portlandonline.com/cgis/me...",False,"{'type': 'Point', 'coordinates': [45.505914, -...","[-122.598196, 45.505914]","{'id': 'ac88a4f17a51c7fc', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [{'text': 'RP20000023885', 'indic...",False,False,low,en,2020-0

In [10]:
df_sample.shape

(290007, 56)

In [11]:
df_sample["geo"] = df_sample["geo"].apply(lambda x: x['coordinates'])

In [12]:
df_sample.head()

,created_at,id,id_str,text,source,truncated,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,user_id,user_id_str,user_name,user_screen_name,user_location,user_url,user_description,user_translator_type,user_protected,user_verified,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_statuses_count,user_created_at,user_utc_offset,user_time_zone,user_geo_enabled,user_lang,user_contributors_enabled,user_is_translator,user_default_profile,user_default_profile_image,user_following,user_follow_request_sent,user_notifications,display_text_range,extended_entities,possibly_sensitive,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,2020-03-19 10:59:57+00:00,1240593849694605312,1240593849694605312,current weather in Burlington: overcast clouds...,"<a href=""http://twitter.com/USACities/cities"" ...",False,"[44.48, -73.21]","[-73.21, 44.48]","{'id': '8f07721fe6eb5c41', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2020-03-19 10:59:57.543,112034664,112034664,Burlington Weather,BurlingtonVT,"Burlington, VT",None,"Weather updates, forecast, warnings and inform...",none,False,False,120,1,14,0,37409,2010-02-07 01:44:27+00:00,NaN,NaN,True,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-19 10:59:59+00:00,1240593857349218309,1240593857349218304,"Flash Flood Warning continues for Nixa MO, Oza...","<a href=""http://www.weather.gov/crh/impact"" re...",False,"[36.82104014, -93.47691589]","[-93.47691589, 36.82104014]","{'id': '2526edd24c06e60c', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2020-03-19 10:59:59.368,600227042,600227042,NWS Springfield,NWSSpringfield,"Springfield, Missouri",http://www.weather.gov/springfield,Official Twitter account for the National Weat...,none,False,True,33227,306,512,2396,17173,2012-06-05 16:24:58+00:00,NaN,NaN,True,NaN,False,False,False,False,NaN,NaN,NaN,"[0, 83]","{'media': [{'id': 1240593855927300098, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN
2,2020-03-19 11:00:00+00:00,1240593862382489600,1240593862382489600,ee69b5dca7d2458d2039dc8f3e12c093a733e7b83d0618...,"<a href=""https://twitter.com/googuns_lulz"" rel...",True,"[27.26026626, -166.01094411]","[-166.01094411, 27.26026626]",None,NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,fr,2020-03-19 11:00:00.568,3048544857,3048544857,GooGuns Lulz,googuns_lulz,(here),https://victorz.ca/bots/googuns,@victor_zheng,none,False,False,259,1,257,0,3973912,2015-02-21 03:26:23+00:00,NaN,NaN,True,NaN,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,{'full_text': 'ee69b5dca7d2458d2039dc8f3e12c09...,NaN,NaN,NaN,NaN
3,2020-03-19 11:00:00+00:00,1240593862659149824,1240593862659149824,"Wind 0.0 mph NNW. Barometer 29.83 in, Rising s...","<a href=""http://sandaysoft.com/"" rel=""nofollow...",False,"[37.92583333, -120.63]","[-120.63, 37.92583333]","{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2020-03-19 11:00:00.634,865368865,865368865,Donald Price,DonaldPrice11,None,None,None,none,False,False,5,24,0,14,8142,2012-10-06 21:52:04+00:00,NaN,NaN,True,NaN,False,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-19 11:00:02+00:00,1240593868438917126,1240593868438917120,"MED - MEDICAL at 6300 SE DIVISION ST, PORT [Po...","<a href=""http://www.portlandonline.com/cgis/me...",False,"[45.505914, -122.598196]","[-122.598196, 45.505914]","{'id': 'ac88a4f17a51c7fc', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [{'text': 'RP20000023885', 'indic...",False,False,low,en,2020-03-19 11:00:02.012,1606472113,1606472113,Mult Co Fire/EMS log,pdxfirelog,"Multnomah County, Oregon",http://www.civicapps.org/datas

In [21]:
df_sample["user_is_translator"].unique

<bound method Series.unique of 0         False
1         False
2         False
3         False
4         False
          ...  
290002    False
290003    False
290004    False
290005    False
290006    False
Name: user_is_translator, Length: 290007, dtype: bool>

In [13]:
df_sample = df_sample.drop(columns=['coordinates', 'source', "contributors", "is_quote_status", "favorited", "retweeted", "filter_level", "user_id", "user_url", "user_translator_type", "user_protected", "user_time_zone","user_contributors_enabled", "user_is_translator", "user_default_profile_image", "user_following","user_follow_request_sent","user_notifications"])
print(df_sample.shape)
df_sample.head()

(290007, 38)


,created_at,id,id_str,text,truncated,geo,place,quote_count,reply_count,retweet_count,favorite_count,entities,lang,timestamp_ms,user_id_str,user_name,user_screen_name,user_location,user_description,user_verified,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_statuses_count,user_created_at,user_utc_offset,user_geo_enabled,user_lang,user_default_profile,display_text_range,extended_entities,possibly_sensitive,extended_tweet,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink
0,2020-03-19 10:59:57+00:00,1240593849694605312,1240593849694605312,current weather in Burlington: overcast clouds...,False,"[44.48, -73.21]","{'id': '8f07721fe6eb5c41', 'url': 'https://api...",0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,2020-03-19 10:59:57.543,112034664,Burlington Weather,BurlingtonVT,"Burlington, VT","Weather updates, forecast, warnings and inform...",False,120,1,14,0,37409,2010-02-07 01:44:27+00:00,NaN,True,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-19 10:59:59+00:00,1240593857349218309,1240593857349218304,"Flash Flood Warning continues for Nixa MO, Oza...",False,"[36.82104014, -93.47691589]","{'id': '2526edd24c06e60c', 'url': 'https://api...",0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,2020-03-19 10:59:59.368,600227042,NWS Springfield,NWSSpringfield,"Springfield, Missouri",Official Twitter account for the National Weat...,True,33227,306,512,2396,17173,2012-06-05 16:24:58+00:00,NaN,True,NaN,False,"[0, 83]","{'media': [{'id': 1240593855927300098, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN
2,2020-03-19 11:00:00+00:00,1240593862382489600,1240593862382489600,ee69b5dca7d2458d2039dc8f3e12c093a733e7b83d0618...,True,"[27.26026626, -166.01094411]",None,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",fr,2020-03-19 11:00:00.568,3048544857,GooGuns Lulz,googuns_lulz,(here),@victor_zheng,False,259,1,257,0,3973912,2015-02-21 03:26:23+00:00,NaN,True,NaN,False,NaN,NaN,NaN,{'full_text': 'ee69b5dca7d2458d2039dc8f3e12c09...,NaN,NaN,NaN,NaN
3,2020-03-19 11:00:00+00:00,1240593862659149824,1240593862659149824,"Wind 0.0 mph NNW. Barometer 29.83 in, Rising s...",False,"[37.92583333, -120.63]","{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api...",0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,2020-03-19 11:00:00.634,865368865,Donald Price,DonaldPrice11,None,None,False,5,24,0,14,8142,2012-10-06 21:52:04+00:00,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-19 11:00:02+00:00,1240593868438917126,1240593868438917120,"MED - MEDICAL at 6300 SE DIVISION ST, PORT [Po...",False,"[45.505914, -122.598196]","{'id': 'ac88a4f17a51c7fc', 'url': 'https://api...",0,0,0,0,"{'hashtags': [{'text': 'RP20000023885', 'indic...",en,2020-03-19 11:00:02.012,1606472113,Mult Co Fire/EMS log,pdxfirelog,"Multnomah County, Oregon","Unofficial automated posting of Portland, Oreg...",False,2844,10,118,0,538176,2013-07-19 17:57:31+00:00,NaN,True,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
testGeo = df_sample['geo']

In [14]:
def readTract(filepath):
    tract = pd.read_csv(filepath)
    tract["County"] = tract["County"].apply(lambda x: unidecode.unidecode(x.replace("St.", "Saint").replace(" city", "").replace("Ste.", "Sainte")))
    tract.head()
    return tract
tract = readTract("../us-census/tract_2017.csv")

In [16]:
def getcountySet(tract):
    countyName = tract[['State','County']]
    countySet = {}
    for i in countyName.groupby("State"):
        countySet[i[0]] = i[1]['County'].values
    return countySet
countySet = getcountySet(tract)

In [17]:
countySet.keys()

dict_keys(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'])

In [20]:
df_sample.iloc[0]

created_at                                         2020-03-19 10:59:57+00:00
id                                                       1240593849694605312
id_str                                                   1240593849694605312
text                       current weather in Burlington: overcast clouds...
truncated                                                              False
geo                                                          [44.48, -73.21]
place                      {'id': '8f07721fe6eb5c41', 'url': 'https://api...
quote_count                                                                0
reply_count                                                                0
retweet_count                                                              0
favorite_count                                                             0
entities                   {'hashtags': [], 'urls': [], 'user_mentions': ...
lang                                                                      en

## GEOCODING !

In [140]:
def geocodingTractTest(testGeo):
    # need to make sure if the geocoding is work
    testGeo = tuple(tuple(x) for x in testGeo)
    rgResults = rg.search(testGeo)
    # Here is filter {"rg item": "tract item"}
    rgFilter = {
        "New York": "New York County",
        "Washington, D.C.": "District of Columbia",
        "De Soto County": "DeSoto County",
        "Annette Island Reserve": "Matanuska-Susitna Borough",
        "Wade Hampton Census Area": "Kusilvak Census Area",
        "Bedford": "Bedford County",
        "La Salle Parish": "LaSalle Parish",
        "Wrangell-Petersburg Census Area": "Wrangell City and Borough",
    }
    counter = 0
    for item in rgResults:
        rgResult = item
        rgState = rgResult["admin1"]
        rgCounty = rgResult["admin2"].replace("City of ", "")
        rgCc = rgResult["cc"]

        if rgCc == "US":
            if rgState in countySet.keys() and rgCounty in countySet[rgState]:
                countinue
            else:
                if rgState in rgFilter.keys():
                    rgState = rgFilter[rgState]

                elif rgCounty in rgFilter.keys():
                    rgCounty = rgFilter[rgCounty]

                else:
                    print("Cannot find State {}".format(rgResult))
                    counter += 1
    return counter


def getrgCountyFilter():
    rgCountyFilter = {
        "New York": "New York County",
        "Washington, D.C.": "District of Columbia",
        "De Soto County": "DeSoto County",
        "Annette Island Reserve": "Matanuska-Susitna Borough",
        "Wade Hampton Census Area": "Kusilvak Census Area",
        "Bedford": "Bedford County",
        "La Salle Parish": "LaSalle Parish",
        "Wrangell-Petersburg Census Area": "Wrangell City and Borough",
        "St. Lawrence County": "Saint Lawrence County",
        "Bronx": "Bronx County",
    }
    return rgCountyFilter


def getrgStateFilter():
    rgStateFilter = {"Washington, D.C.": "District of Columbia"}
    return rgStateFilter


def formatRg(rgCounty, rgState):
    c, s = "", ""
    if str(rgState) == "Washington, D.C.":
        s = "District of Columbia"
        c = "District of Columbia"
    elif str(rgState) == "New York":
        s = "New York"
        c = "New York County"
    else:
        s = rgState
        c = rgCounty
    return c, s


def addgeocodingToTract(tract, df_tweets):
    rgCounty, rgState = "", ""
    countySet = getcountySet(tract)
    tweetsGeo = df_tweets["geo"]
    tweetsGeo = tuple(tuple(x) for x in tweetsGeo)
    rgResults = rg.search(tweetsGeo)
    # Here is filter {"rg item": "tract item"}

    rgStateFilter = getrgStateFilter()
    rgCountyFilter = getrgCountyFilter()

    counter = 0
    # counter is used to count the error geocoding
    countyId = []
    for index, item in zip(df_tweets.index, rgResults):

        rgResult = item

        rgState = copy.deepcopy(rgResult["admin1"])
        rgCounty = copy.deepcopy(rgResult["admin2"].replace("City of ", ""))
        rgCc = rgResult["cc"]

        if rgCc == "US":
            if not rgCounty or not rgState:
                rgCounty, rgState = formatRg(rgCounty, rgState)
                # print(rgCounty, rgState)

            if rgState in countySet.keys() and rgCounty in countySet[rgState]:
                cId = tract.query(
                    'State == "{}" and County == "{}"'.format(rgState, rgCounty)
                ).CountyId.values
                countyId.append(cId[0])
            else:
                if rgState in rgStateFilter.keys():
                    rgState = rgStateFilter[rgState]
                    cId = tract.query(
                        'State == "{}" and County == "{}"'.format(rgState, rgCounty)
                    ).CountyId.values
                    if len(cId) == 0:
                        print(rgResult, rgCounty)
                        return
                    countyId.append(cId[0])
                elif rgCounty in rgCountyFilter.keys():
                    rgCounty = rgCountyFilter[rgCounty]
                    cId = tract.query(
                        'State == "{}" and County == "{}"'.format(rgState, rgCounty)
                    ).CountyId.values
                    if len(cId) == 0:
                        print(rgResult, rgCounty)
                        return
                    countyId.append(cId[0])
                else:
                    counter += 1
                    print("Cannot find State {}".format(rgResult))
        else:
            # if the tweets is not in US
            countyId.append(np.nan)

    return countyId

In [141]:
countyId = addgeocodingToTract(tract, df_sample)

In [145]:
df_sample["CountyId"] = [str(x) for x in countyId]

In [127]:
tract.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,0.9,0.0,41016,55317,2838,27824,2024,13.7,20.1,35.3,18.0,23.2,8.1,15.4,86.0,9.6,0.1,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,0.7,0.0,155376,52562,1348,29364,735,11.8,16.1,35.7,18.2,25.6,9.7,10.8,84.7,7.6,0.1,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,0.6,0.0,20269,33368,2551,17561,798,27.2,44.9,25.0,16.8,22.6,11.5,24.1,83.4,11.1,0.3,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,0.0,0.0,17662,43404,3431,20911,1889,15.2,26.6,24.4,17.6,19.7,15.9,22.4,86.4,9.5,0.7,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,0.1,0.0,42513,47412,2630,22021,850,15.6,25.4,28.5,12.9,23.3,15.8,19.5,86.8,10.2,0.1,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [99]:
unidecode.unidecode("Doña")

'Dona'

In [33]:
tract.query('State == "{}" and County == "{}"'.format("Alabama", "Autauga County")).CountyId.values[0]

1001

In [ ]:
df_sample.to_csv("tweets-with-geocoding.csv", index=False)

In [157]:
df_sample.shape

(290007, 39)